# Employment - Occupation

The data is sourced from BLS' Occupational Employment and Wage Statistics (OEWS) program database. The data link is the All OEWS data file for MSAs from 2003 to 2020. The data is updated on an yearly basis in May.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from os import *
import sys
sys.path.append("../")
import zipfile
import urllib.request
from shutil import copyfile
import shutil

In [3]:
# Go to the input folder
def change_dir_to_input(newpath):
    get_path = sys.path[0].split("\\")      
    del get_path[-1]                        
    cwd = "input\\" + str(newpath)                          
    get_path.append(cwd)                    
    path = "\\".join(get_path)              
    os.chdir(path)    

In [4]:
# Get input files from 2003 to 2020
change_dir_to_input('')
# os.mkdir('occupation_orig')
for i in range(5,21):
    change_dir_to_input('occupation_orig')
    if i in range(5,10):
        url = 'https://www.bls.gov/oes/special.requests/oesm0'+ str(i) + 'ma.zip'
        urllib.request.urlretrieve(url, 'oesm0'+ str(i) + 'ma.zip')
        zip_filename = 'oesm0'+ str(i) + 'ma.zip'
    else:
        url = 'https://www.bls.gov/oes/special.requests/oesm'+ str(i) + 'ma.zip'
        urllib.request.urlretrieve(url, 'oesm'+ str(i) + 'ma.zip')
        zip_filename = 'oesm'+ str(i) + 'ma.zip'
    zf = zipfile.ZipFile(zip_filename)
    if i in range(5,14):
        files = [i.filename  for i in zf.infolist()]
        files_to_del = filter( lambda f: f.startswith('a') or f.startswith('BOS') or f.startswith('field') or f.startswith('file'), zf.namelist())
        for k in list(files_to_del):
            files.remove(k)
        df = pd.DataFrame()
        for j in files:
            li = pd.read_excel(zf.open(j))
            df = df.append(li, ignore_index=True)
    elif i in range(14, 21):
        df = pd.read_excel(zf.open('oesm'+ str(i) + 'ma/MSA_M20'+ str(i) +'_dl.xlsx'))
    df.columns = map(str.upper, df.columns)
    if 'OCC_GROUP' in list(df.columns):
        df = df.rename(columns={'OCC_GROUP': 'O_GROUP'})
    if 'GROUP' in list(df.columns):
        df = df.rename(columns={'GROUP': 'O_GROUP'})
    if 'AREA_NAME' in list(df.columns):
        df = df.rename(columns={'AREA_NAME': 'AREA_TITLE'})
    if 'PRIM_STATE' not in list(df.columns):
        df['PRIM_STATE'] = df['AREA_TITLE'].str.split(', ').str[-1]
    lst = ['MSA','PMSA']
    df['AREA_TITLE'] = df['AREA_TITLE'].replace(lst,'', regex=True)
    df['AREA_TITLE'] = df['AREA_TITLE'].str.strip()
    df = df[df.O_GROUP.str.contains("major",na=False)]
    df = df.drop(['NAICS','NAICS_TITLE','I_GROUP','OWN_CODE','PCT_TOTAL','ANNUAL','HOURLY','OCC_TITLE'],axis=1,errors='ignore')
    change_dir_to_input('occupation_out')
    if i in range(3,10):
        df['year'] = '200' + str(i)
        df.to_csv('MSA200'+str(i)+'.csv')
    else:
        df['year'] = '20' + str(i)
        df.to_csv('MSA20'+str(i)+'.csv')

In [6]:
change_dir_to_input('occupation_out')
li = []
for i in range(5,21):
    if i in range(3,10):
        df = pd.read_csv('MSA200'+str(i)+'.csv')
    else:
        df = pd.read_csv('MSA20'+str(i)+'.csv')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [7]:
# Go to the Output folder
get_path = sys.path[0].split("\\")
del get_path[-1]
cwd = "output"
get_path.append(cwd)
path = "\\".join(get_path)
os.chdir(path)

In [8]:
# Convert the output to csv
frame.to_csv('occupations.csv')

In [9]:
df = pd.read_csv('occupations.csv')
df.columns

C:\Users\himan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (23,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Unnamed: 0', 'Unnamed: 0.1', 'PRIM_STATE', 'AREA', 'AREA_TITLE',
       'OCC_CODE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN', 'A_MEAN',
       'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90',
       'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'year',
       'JOBS_1000', 'LOC QUOTIENT', 'AREA_TYPE', 'LOC_QUOTIENT'],
      dtype='object')

In [10]:
# Drop irrelevant columns
df.drop(df.columns.difference(['PRIM_STATE', 'AREA','AREA_TITLE','OCC_CODE','O_GROUP','TOT_EMP','year']), 1, inplace=True)
df

,PRIM_STATE,AREA,AREA_TITLE,OCC_CODE,O_GROUP,TOT_EMP,year
0,AK,11260,"Anchorage, AK",11-0000,major,12880,2005
1,AK,11260,"Anchorage, AK",13-0000,major,5880,2005
2,AK,11260,"Anchorage, AK",15-0000,major,2590,2005
3,AK,11260,"Anchorage, AK",17-0000,major,4360,2005
4,AK,11260,"Anchorage, AK",19-0000,major,2820,2005
...,...,...,...,...,...,...,...
140570,MA,79600,"Worcester, MA-CT",45-0000,major,330,2020
140571,MA,79600,"Worcester, MA-CT",47-0000,major,9110,2020
140572,MA,79600,"Worcester, MA-CT",49-0000,major,9160,2020
140573,MA,79600,"Worcester, MA-CT",51-0000,major,18430,2020


In [11]:
# Filter Puerto Rico
df = df[df.PRIM_STATE != 'PR']
df

,PRIM_STATE,AREA,AREA_TITLE,OCC_CODE,O_GROUP,TOT_EMP,year
0,AK,11260,"Anchorage, AK",11-0000,major,12880,2005
1,AK,11260,"Anchorage, AK",13-0000,major,5880,2005
2,AK,11260,"Anchorage, AK",15-0000,major,2590,2005
3,AK,11260,"Anchorage, AK",17-0000,major,4360,2005
4,AK,11260,"Anchorage, AK",19-0000,major,2820,2005
...,...,...,...,...,...,...,...
140570,MA,79600,"Worcester, MA-CT",45-0000,major,330,2020
140571,MA,79600,"Worcester, MA-CT",47-0000,major,9110,2020
140572,MA,79600,"Worcester, MA-CT",49-0000,major,9160,2020
140573,MA,79600,"Worcester, MA-CT",51-0000,major,18430,2020


In [19]:
df = df.sort_values('AREA_TITLE')

In [20]:
# Parse years as columns
df3 = df.set_index(['PRIM_STATE', 'AREA','AREA_TITLE','OCC_CODE','O_GROUP','year'])['TOT_EMP'].unstack()
df3

year                                              2005   2006   2007   2008  \
PRIM_STATE AREA  AREA_TITLE    OCC_CODE O_GROUP                               
AK         11260 Anchorage, AK 11-0000  major    12880  13200  13230  12430   
                               13-0000  major     5880   6200   5860   6360   
                               15-0000  major     2590   2510   2500   2760   
                               17-0000  major     4360   4290   4520   4810   
                               19-0000  major     2820   2530   2550   2840   
...                                                ...    ...    ...    ...   
WY         16940 Cheyenne, WY  45-0000  major      NaN    NaN    NaN    NaN   
                               47-0000  major     2260   2750   2560   2950   
                               49-0000  major     1930   1840   2100   1960   
                               51-0000  major     1430   1540   1690   1740   
                               53-0000  major     3310   3420   2670   3990   

year                                              2009   2010   2011   2012  \
PRIM_STATE AREA  AREA_TITLE    OCC_CODE O_GROUP                               
AK         11260 Anchorage, AK 11-0000  major    11910  10720  10350  10040   
                               13-0000  major     6940   7340   7270   7900   
                               15-0000  major     3110   3320   3390   3500   
                               17-0000  major     5370   5530   5440   5530   
                               19-0000  major     2830   2690   2750   3000   
...                                                ...    ...    ...    ...   
WY         16940 Cheyenne, WY  45-0000  major      NaN    NaN    NaN    NaN   
                               47-0000  major     2720   2780   2600   2770   
                               49-0000  major     2100   2190   2300   2380   
                               51-0000  major     1420   1110   1200   1390   
                               53-0000  major     3830   3510   3670   3570   

year                                              2013   2014   2015  2016  \
PRIM_STATE AREA  AREA_TITLE    OCC_CODE O_GROUP                              
AK         11260 Anchorage, AK 11-0000  major    10370  10000  10090  9820   
                               13-0000  major     8180   8320   8980  9190   
                               15-0000  major     3360   3190   3350  3370   
                               17-0000  major     5260   5140   5270  5070   
                               19-0000  major     3010   2870   2710  2590   
...                                                ...    ...    ...   ...   
WY         16940 Cheyenne, WY  45-0000  major      NaN    NaN     30    40   
                               47-0000  major     3220   3000   3330  3110   
                               49-0000  major     2300   2210   2180  2090   
                               51-0000  major     1410   1300   1130  1120   
                               53-0000  major     3580   4110   4390  4070   

year                                             2017   2018   2019   2020  
PRIM_STATE AREA  AREA_TITLE    OCC_CODE O_GROUP                             
AK         11260 Anchorage, AK 11-0000  major    9950  10610  11830  12000  
                               13-0000  major    8940   8000   7920   7780  
                               15-0000  major    3570   3300   3280   2980  
                               17-0000  major    4820   4610   4440   4390  
                               19-0000  major    2480   2390   2930   3050  
...                                               ...    ...    ...    ...  
WY         16940 Cheyenne, WY  45-0000  major      50     40     30    NaN  
                               47-0000  major    2930   2960   3120   3190  
                               49-0000  major    2300   2180   2250   2110  
                               51-0000  major    1240   1260   1560   1570  
     

In [14]:
df3.to_csv('occs_by_year.csv')   # naming convention 